## 통계학 실험 9장 과제 

####  1.  (run10samp.txt) 자료를 이용한 예제

주어진 자료는 2012년 Washington, DC에서 열렸던 Cherry Blossom 10 mile run 경기에서 완주를 한 선수 100명의 자료이다. 주요 변수에 대한 설명은 다음과 같다.

|  변수명       |        설명              |
|--------|:-------------------------------------|
| time |  10 마일 달리기 완주 기록 (분)       |
| age | 선수 나이   |
| gender | 성별 (M=남성, F=여성) |
| state | 출신 지역 |


In [3]:
import lzma
import pandas as pd

(data := pd.read_csv(lzma.open('run10samp.txt.xz'), sep=' '))

,time,age,gender,state
1,88.31,59,M,MD
2,100.67,32,M,VA
3,109.52,33,F,VA
4,87.11,52,M,PA
5,73.02,40,M,DC
...,...,...,...,...
96,90.47,35,F,VA
97,94.32,28,F,VA
98,96.19,32,F,DC
99,104.31,32,F,MD


(1) 성별에 따라 완주시간에 차이가 있는지를 일원배치법을 이용하여 유의수준 5%에서 검정해보자. 가설을 세우고 검정통계량(F값), P_value를 쓰고 귀무가설 기각여부를 쓰시오.

(2) QQ-plot을 이용하여 성별에 따른 완주시간의 분포가 정규분포를 따르는지 확인하시오.


(3) 등분산을 가정하고 성별에 따른 완주시간에 차이가 있는지를 t-test를 이용하여 유의수준 5%에서 검정해보자. 가설을 세우고 검정통계량값(t값), P_value를 쓰고 귀무가설 기각여부를 쓰시오.

(4) (1)에서의 검정통계량 값과 (2)에서의 검정통계량 값은 어떠한 관계가 있고, 왜 이러한 관계가 있는지 쓰시오.

(5) run10Samp의 age를 정확한 나이가 아닌 위의 문제와 같이 연령대로 나누어 볼 것이다. run10Samp의 age열을 나이대로 바꾸시오.(58->5,42->4) \ 나이대와 성별에 따라 완주시간에 차이가 있는지를 반복이 없는 이원배치법을 이용하여 검정하고자 한다. 귀무가설과 대립가설을 쓰시오.

(6) 성별과 연령이 완주시간에 영향을 미치는지 유의수준 5%에서 검정해보자. 검정통계량(F값), P_value를 쓰고 귀무가설 기각여부를 쓰시오.

(7) 성별과 연령에 대한 상호작용 유무를 interaction plot을 통해 확인해보시오